In [ ]:
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
test = test['excerpt']

In [ ]:
text_token = []
stemmer = PorterStemmer()
for text in train['excerpt']:
    word_token = []
    sentences = sent_tokenize(text)
    for sentence in sentences:
        words = word_tokenize(sentence)
        for word in words:
            if word not in stopwords.words('english'):
                word = stemmer.stem(word)
                word_token.append(word)
    text_token.append(word_token)

In [ ]:
test_text_token = []
stemmer = PorterStemmer()
for text in test:
    word_token = []
    sentences = sent_tokenize(text)
    for sentence in sentences:
        words = word_tokenize(sentence)
        for word in words:
            if word not in stopwords.words('english'):
                word = stemmer.stem(word)
                word_token.append(word)
    test_text_token.append(word_token)

In [ ]:
t = Tokenizer()
t.fit_on_texts(text_token)
vocab_size = len(t.word_index) + 1
X_encoded = t.texts_to_sequences(text_token)
max_len=max(len(l) for l in X_encoded)
X_test_encoded = t.texts_to_sequences(test_text_token)

X_train=pad_sequences(X_encoded, maxlen=max_len, padding='post')
X_test = pad_sequences(X_test_encoded, maxlen=max_len, padding='post')
y_train=np.array(train['target'])

X_train.shape, X_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from keras.layers import Dense, LSTM, InputLayer, Bidirectional,Dropout

model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len))
model.add(Bidirectional(LSTM(256, return_sequences=False), input_shape=X_train.shape[1:]))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=15, verbose=1,validation_split=0.4,batch_size=128)

In [ ]:
predict = model.predict(X_test)
submit = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
submit['target'] = predict
submit.to_csv('submission.csv',index=None)